## 시드 고정

In [ ]:
import os
import random
import numpy as np
import torch

# 시드설정
SEED = 123

def seed_everything(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(SEED)

## 샘플 예제파일 다운로드

In [ ]:
import urllib

# bbc-text.csv 데이터셋 다운로드
url = 'https://storage.googleapis.com/download.tensorflow.org/data/bbc-text.csv'
urllib.request.urlretrieve(url, 'bbc-text.csv')

## 데이터 로드

In [ ]:
import json
from tqdm import tqdm
import numpy as np
import pandas as pd
    
# 데이터프레임을 로드 합니다.
df = pd.read_csv('bbc-text.csv')
df.head()

## 토큰화 (Word Tokenization)

- get_tokenizer로 토크나이저 생성
- `basic_english`, `spacy`, `revtok`, `subword` 등 지정이 가능하나, 몇몇 토크나이저는 추가 라이브러리 설치가 필요합니다.

In [ ]:
from torchtext.data.utils import get_tokenizer

# 토큰 생성
tokenizer = # 코드작성

## 단어사전 생성

In [ ]:
from torchtext.vocab import build_vocab_from_iterator

def yield_tokens(sentences):
    for text in sentences:
        yield tokenizer(text)

`build_vocab_from_iterator` 를 활용하여 단어 사전을 생성합니다.

- `min_freq`: 최소 빈도의 토큰의 개수를 입력합니다.
- `max_tokens`: 최대 빈도 토큰의 수를 한정합니다. 빈도수 기준으로 산정합니다.

In [ ]:
vocab = # 코드작성
vocab.set_default_index(vocab['<UNK>'])

## 단어사전의 개수 출력

In [ ]:
# 전체 단어사전의 개수 출력
len(vocab)

## 라벨 맵 생성 (문자 -> 숫자 변환)

In [ ]:
{v:i for i, v in enumerate(df['category'].value_counts().keys())}

In [ ]:
label_map = {
    'sport': 0, 
    'business': 1, 
    'politics': 2, 
    'tech': 3, 
    'entertainment': 4
}

In [ ]:
df['category_num'] = df['category'].map(label_map)

## Dataset 분할

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = # 코드작성

## Dataset 생성

In [ ]:
from torch.utils.data import DataLoader, Dataset


class CustomDataset(Dataset):
    def __init__(self, texts, labels, vocab, tokenizer):
        super().__init__()
        # 코드작성
        
        
    def __len__(self):
        return # 코드작성
        
    def __getitem__(self, idx):
        text = # 코드작성
        label = # 코드작성
        return # 코드작성

In [ ]:
# Custom Dataset 생성
train_ds = # 코드작성
valid_ds = # 코드작성

In [ ]:
# 1개의 데이터 추출
text, label = next(iter(train_ds))
len(text), label

## DataLoader 생성

In [ ]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
def collate_batch(batch, max_sequence_length):
    label_list, text_list = [], []
    
    for text, label in batch:
        # 최대 문장길이를 넘어가는 단어는 제거합니다.
        processed_text = # 코드작성
        text_list.# 코드작성
        label_list.# 코드작성
    
    label_list = # 코드작성
    
    # padding을 주어 짧은 문장에 대한 길이를 맞춥니다.
    text_list = # 코드작성
    
    return # 코드작성

In [ ]:
# 한 문장에 최대 포함하는 단어의 개수를 지정합니다. (예시. 200 단어)
MAX_SEQUENCE_LENGTH = # 코드작성

train_loader = # 코드작성

valid_loader = # 코드작성

In [ ]:
x, y = next(iter(train_loader))
x = x.to(device)
y = y.to(device)

x.shape, y.shape
# (batch_size, seq_length), (batch_size)

## 모델

In [ ]:
from tqdm import tqdm  # Progress Bar 출력
import numpy as np
import torch.nn as nn
import torch.optim as optim


class TextClassificationModel(nn.Module):
    def __init__(self, num_classes, vocab_size, embedding_dim, hidden_size, num_layers, bidirectional=True, drop_prob=0.1):
        super(TextClassificationModel, self).__init__()
        self.num_classes = # 코드작성
        self.vocab_size = # 코드작성
        self.embedding_dim = # 코드작성
        self.hidden_size = # 코드작성
        self.num_layers = # 코드작성
        self.bidirectional = # 코드작성
        
        self.embedding = # 코드작성
        
        self.lstm = # 코드작성
        
        self.dropout = # 코드작성
        
        self.relu = # 코드작성
        
        self.fc = # 코드작성
        self.output = # 코드작성
        
    def init_hidden_and_cell_state(self, batch_size, device):
        # LSTM 입력시 초기 Cell 에 대한 가중치 초기화를 진행합니다.
        # (num_layers*bidirectional, batch_size, hidden_size)
        self.hidden_and_cell = (
            torch.zeros(self.num_layers*self.bidirectional, batch_size, self.hidden_size).to(device),
            torch.zeros(self.num_layers*self.bidirectional, batch_size, self.hidden_size).to(device),
        )
        
    def forward(self, x):
        x = self.embedding(x)
        output, (h, c) = self.lstm(x, self.hidden_and_cell)
        # (batch_size, seq_length, hidden_size*bidirectional)
        # last sequence 의 (batch_size, hidden_size*bidirectional)
        h = output[:, -1, :]
        o = self.dropout(h)
        o = self.relu(self.fc(o))
        o = self.dropout(o)
        return self.output(o)

## 모델 생성

In [ ]:
config = {
    'num_classes': # 코드작성, 
    'vocab_size': # 코드작성,
    'embedding_dim': # 코드작성 
    'hidden_size': # 코드작성
    'num_layers': # 코드작성 
    'bidirectional': # 코드작성
}

model = TextClassificationModel(**config)
model.to(device)

## 손실함수 및 옵티마이저 정의

In [ ]:
# loss 정의: CrossEntropyLoss
loss_fn = # 코드작성

# 옵티마이저 정의: bert.paramters()와 learning_rate 설정
optimizer = # 코드작성

In [ ]:
def model_train(model, data_loader, loss_fn, optimizer, device):
    # 모델을 훈련모드로 설정합니다. training mode 일 때 Gradient 가 업데이트 됩니다. 반드시 train()으로 모드 변경을 해야 합니다.
    model.train()
    
    # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
    running_loss = 0
    corr = 0
    counts = 0
    
    # 예쁘게 Progress Bar를 출력하면서 훈련 상태를 모니터링 하기 위하여 tqdm으로 래핑합니다.
    prograss_bar = tqdm(data_loader, unit='batch', total=len(data_loader), mininterval=1)
    
    # mini-batch 학습을 시작합니다.
    for idx, (txt, lbl) in enumerate(prograss_bar):
        # txt, lbl 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
        txt = txt.to(device)
        lbl = lbl.to(device)
        
        # 누적 Gradient를 초기화 합니다.
        optimizer.zero_grad()
        
        # LSTM Weight 초기화
        model.# 코드작성
        
        # Forward Propagation을 진행하여 결과를 얻습니다.
        output = # 코드작성
        
        # 손실함수에 output, lbl 값을 대입하여 손실을 계산합니다.
        loss = # 코드작성
        
        # 오차역전파(Back Propagation)을 진행하여 미분 값을 계산합니다.
        loss.# 코드작성
        
        # 계산된 Gradient를 업데이트 합니다.
        optimizer.# 코드작성
        
        # Probability Max index 를 구합니다.
        output = output.# 코드작성
        
        # 정답 개수를 구합니다.
        corr += (output == lbl).sum().item()
        counts += len(lbl)
        
        # batch 별 loss 계산하여 누적합을 구합니다.
        running_loss += loss.item()
        
        # 프로그레스바에 학습 상황 업데이트
        prograss_bar.set_description(f"training loss: {running_loss/(idx+1):.5f}, training accuracy: {corr / counts:.5f}")
        
    # 누적된 정답수를 전체 개수로 나누어 주면 정확도가 산출됩니다.
    acc = corr / len(data_loader.dataset)
    
    # 평균 손실(loss)과 정확도를 반환합니다.
    # train_loss, train_acc
    return running_loss / len(data_loader), acc

In [ ]:
def model_evaluate(model, data_loader, loss_fn, device):
    # model.eval()은 모델을 평가모드로 설정을 바꾸어 줍니다. 
    # dropout과 같은 layer의 역할 변경을 위하여 evaluation 진행시 꼭 필요한 절차 입니다.
    model.eval()
    
    # Gradient가 업데이트 되는 것을 방지 하기 위하여 반드시 필요합니다.
    with torch.no_grad():
        # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
        corr = 0
        running_loss = 0
        
        # 배치별 evaluation을 진행합니다.
        for txt, lbl in data_loader:
            # txt, lbl 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
            txt = txt.to(device)
            lbl = lbl.to(device)
            
            # LSTM Weight 초기화
            model.# 코드작성
    
            # 모델에 Forward Propagation을 하여 결과를 도출합니다.
            output = # 코드작성
            
            # 검증 손실을 구합니다.
            loss = # 코드작성
            
            # Probability Max index 를 구합니다.
            output = output.# 코드작성
            
            # 정답 개수를 구합니다.
            corr += (output == lbl).sum().item()
            
            # batch 별 loss 계산하여 누적합을 구합니다.
            running_loss += loss.item()
        
        # validation 정확도를 계산합니다.
        # 누적한 정답숫자를 전체 데이터셋의 숫자로 나누어 최종 accuracy를 산출합니다.
        acc = corr / len(data_loader.dataset)
        
        # 결과를 반환합니다.
        # val_loss, val_acc
        return running_loss / len(data_loader), acc

In [ ]:
# 최대 Epoch을 지정합니다.
num_epochs = 50

# checkpoint로 저장할 모델의 이름을 정의 합니다.
model_name = 'BBC-Text-CLF-LSTM'

min_loss = np.inf

# Epoch 별 훈련 및 검증을 수행합니다.
for epoch in range(num_epochs):
    # Model Training
    # 훈련 손실과 정확도를 반환 받습니다.
    train_loss, train_acc = model_train(model, train_loader, loss_fn, optimizer, device)

    # 검증 손실과 검증 정확도를 반환 받습니다.
    val_loss, val_acc = model_evaluate(model, valid_loader, loss_fn, device)   
    
    # val_loss 가 개선되었다면 min_loss를 갱신하고 model의 가중치(weights)를 저장합니다.
    if val_loss < min_loss:
        print(f'[INFO] val_loss has been improved from {min_loss:.5f} to {val_loss:.5f}. Saving Model!')
        min_loss = val_loss
        torch.save(model.state_dict(), f'{model_name}.pth')
    
    # Epoch 별 결과를 출력합니다.
    print(f'epoch {epoch+1:02d}, loss: {train_loss:.5f}, acc: {train_acc:.5f}, val_loss: {val_loss:.5f}, val_accuracy: {val_acc:.5f}')

## 저장한 가중치 로드

In [ ]:
model.# 코드작성

## 최종 검증 손실 및 정확도 출력

In [ ]:
model.eval()

with torch.no_grad():
    val_loss, val_acc = model_evaluate(model, valid_loader, loss_fn, device)
    
    print(f'loss: {val_loss:.5f}, accuracy: {val_acc:.5f}')